<a href="https://colab.research.google.com/github/Gabrielyuug/YUUG-IA---Alura-Google/blob/main/projeto_yuug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install -U -q google-generativeai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [26]:
!pip install -q google-cloud-videointelligence google-auth files pytube

Instalando o SDK do Google

In [27]:
import google.generativeai as genai

GOOGLE_API_KEY=('insira sua api key aqui')

genai.configure(api_key=GOOGLE_API_KEY)

Listar os modelos disponíveis

In [28]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [29]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.8,
}

safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

Limitando o modelo

In [30]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [31]:
%%time
response = model.generate_content("Você é um analista de CS:GO")

CPU times: user 87.1 ms, sys: 10.2 ms, total: 97.3 ms
Wall time: 6.49 s


In [32]:
from IPython.display import Markdown as md
md(response.text)

Correto! Sou capaz de analisar partidas e jogadores de CS:GO, fornecendo insights táticos e estratégicos. 

Para me ajudar a analisar, me diga:

* **O que você gostaria de analisar?** Uma partida específica? Um jogador em particular? Uma equipe? Um mapa? Uma estratégia? 
* **Quais informações você possui?** Demos da partida, estatísticas dos jogadores, vídeos, etc. Quanto mais informações você fornecer, mais completa será minha análise.

Com base nas suas respostas, posso ajudá-lo a:

* **Identificar pontos fortes e fracos** de jogadores e equipes.
* **Analisar a economia** durante a partida.
* **Avaliar a tomada de decisão** em momentos cruciais.
* **Desvendar as estratégias** utilizadas por ambos os lados.
* **Sugerir melhorias** para o seu jogo individual ou em equipe.

Estou pronto para começar a análise! 😊 


In [33]:
import cv2
import os
import shutil

# Create or cleanup existing extracted image frames directory.
FRAME_EXTRACTION_DIRECTORY = "/content/frames"
FRAME_PREFIX = "_frame"
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
  print(f"Extracting {video_file_path} at {60} frame per second. This might take a bit...")
  create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
  vidcap = cv2.VideoCapture(video_file_path)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  frame_duration = 1 / fps  # Time interval between frames (in seconds)
  output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
  frame_count = 0
  count = 0
  while vidcap.isOpened():
      success, frame = vidcap.read()
      if not success: # End of video
          break
      if int(count / fps) == frame_count: # Extract a frame every second
          min = frame_count // 60
          sec = frame_count % 60
          time_string = f"{min:02d}:{sec:02d}"
          image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
          output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
          cv2.imwrite(output_filename, frame)
          frame_count += 1
      count += 1
  vidcap.release() # Release the capture object\n",
  print(f"Completed video frame extraction!\n\nExtracted: {frame_count} frames")

In [34]:
import os

class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

def upload_frames():
  # Process each frame in the output directory
  files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
  files = sorted(files)
  files_to_upload = []
  for file in files:
    files_to_upload.append(
        File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

  # Upload the files to the API
  # Only upload a 10 second slice of files to reduce upload time.
  # Change full_video to True to upload the whole video.
  full_video = True

  uploaded_files = []
  print(f'Uploading {len(files_to_upload) if full_video else 10} files. This might take a bit...')

  for file in files_to_upload if full_video else files_to_upload[40:50]:
    #print(f'Uploading: {file.file_path}...')
    response = genai.upload_file(path=file.file_path)
    file.set_file_response(response)
    uploaded_files.append(file)

  print(f"Completed file uploads!\nUploaded: {len(uploaded_files)} files")
  return uploaded_files

def list_files(uploaded_files):
# List files uploaded in the API
  for n, f in zip(range(len(uploaded_files)), genai.list_files()):
    print(f.uri)


In [35]:
# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

In [ ]:
from pytube import YouTube

video_url = input("Insira a URL da demo ou jogada do youtube: ")
video_name = "video_sample.mp4"

while video_url != "fim":
  # Get video from yt
  print("\nNow just wait, your answer will arrive faster than the speed of light, or not...")
  YouTube(video_url).streams.get_highest_resolution().download(filename=video_name)

  # Get frames
  extract_frame_from_video(video_name)

  # Make the LLM request.
  # list_files(upload_frames()) # DEBUG ONLY
  uploaded_files = upload_frames()

  question = input("\nQual a pergunta sobre a jogada: ")
  while question != "fim":
    request = make_request(question, uploaded_files)
    response = model.generate_content(request, request_options={"timeout": 600})
    print(f"{response.text}\n")
    question = input("Qual a pergunta sobre a jogada: ")

  # Do again while prompt != "fim"
  video_url = input("Insira a URL da demo ou jogada do youtube: ")

Insira a URL da demo ou jogada do youtube: https://www.youtube.com/watch?v=RaI82xO6yjs

Now just wait, your answer will arrive faster than the speed of light, or not...
Extracting video_sample.mp4 at 30 frame per second. This might take a bit...
Completed video frame extraction!

Extracted: 53 frames
Uploading 53 files. This might take a bit...
Completed file uploads!
Uploaded: 53 files

Qual a pergunta sobre a jogada: O que acontece nesse video
Um jogador de Counter-Strike: Global Offensive chamado m0NESY faz uma jogada incrível para ganhar a rodada para seu time, G2 Esports. Ele mata dois jogadores inimigos enquanto corre para plantar a bomba, em seguida planta a bomba e elimina o último jogador inimigo com um tiro na cabeça através de uma parede. Seus companheiros de equipe estão extasiados com a jogada, e você pode ver suas reações na câmera.

Qual a pergunta sobre a jogada: Quem é monesy
m0NESY é um jogador profissional de Counter-Strike: Global Offensive para a G2 Esports.

Qual 